### Install Dependencies
Make sure you have the required dependencies installed. You can do this by running the following command in your terminal:

In [1]:
%pip install --only-binary=:all: tiktoken
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install required libraries
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
    langchain_community \
    unstructured[md]==0.17.* \
    langgraph==0.4.* \
    websockets==15.0.*

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import json
import asyncio
import requests
import websockets
from dotenv import load_dotenv
import nest_asyncio

# Allow asyncio.run in Jupyter
nest_asyncio.apply()

from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

# ------------------ Load Environment ------------------
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")
if openai_api_key:
    print("------------------- Loaded Key -------------------")

# ------------------ Load OpenAPI Spec ------------------
root = "http://localhost:3000"
api_spec_url = f"{root}/api/docs/openapi.json"
response = requests.get(api_spec_url)
data = response.json()
data['servers'] = [{'url': root}]
openapi_spec = reduce_openapi_spec(data, dereference=False)

# ------------------ Setup LLM and Requests ------------------
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)
requests_wrapper = RequestsWrapper()

# ------------------ Create Base Agent ------------------
agent = planner.create_openapi_agent(
    api_spec=openapi_spec,
    requests_wrapper=requests_wrapper,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    handle_parsing_errors=True,
    allow_operations=['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
)

# ------------------ Per-Ticket Context ------------------
ticket_contexts = {}

def get_ticket_context(ticket_id):
    if ticket_id not in ticket_contexts:
        ticket_contexts[ticket_id] = []
    return ticket_contexts[ticket_id]

def clear_ticket_context(ticket_id):
    ticket_contexts.pop(ticket_id, None)

async def run_agent(ticket_id, action_description, instructions):
    """
    Run agent asynchronously with per-ticket context.
    Each ticket sees its previous steps.
    """
    context = get_ticket_context(ticket_id)
    previous_steps = "\n".join(context)
    prompt = f"Previous steps:\n{previous_steps}\n\nCurrent task:\n{instructions}"
    print(f"{action_description} for ticket: {ticket_id}")
    
    # Run the agent
    result = await asyncio.to_thread(agent.invoke, prompt.strip())
    
    # Convert result to string before storing
    context.append(str(result))  # or json.dumps(result) if you want JSON format
    
    return result


# --- Ticket Decision Functions (All in One Block) ---

async def determine_category(ticket_id):
    await run_agent(
        ticket_id,
        "Categorizing ticket",
        f"""
Update the ticket's category.

POST /api/tickets/{ticket_id}/category
Send ONLY this JSON body:
{{
  "category": "<chosen_category>"
}}

Valid categories:
- Mechanical
- Quality
- Maintenance
- Technical
- Awaiting Details
"""
    )


async def determine_category(ticket_id):
    await run_agent(
        ticket_id,
        "Categorizing ticket",
        f"""
Update the ticket's ({ticket_id}) category.

POST /api/tickets/{ticket_id}/category
Send ONLY this JSON body:
{{
  "category": "<chosen_category>"
}}

Valid categories:
- Mechanical
- Quality
- Maintenance
- Technical
- Awaiting Details

If insufficient info exists, choose "Awaiting Details".
"""
    )


async def determine_priority(ticket_id):
    await run_agent(
        ticket_id,
        "Determining ticket priority",
        f"""
Update the ticket's ({ticket_id})  priority.

POST /api/tickets/{ticket_id}/priority
Send ONLY this JSON body:
{{
  "priority": "<chosen_priority>"
}}

Valid priorities:
- High
- Medium
- Low

If category is "Awaiting Details", always choose "Low".
"""
    )


async def determine_response(ticket_id):
    await run_agent(
        ticket_id,
        "Checking for automatic response",
        f"""
Retrieve full ticket ({ticket_id})  details:

GET /api/tickets/{ticket_id}

If category == "Awaiting Details":
  → POST a response requesting more info.

Send JSON body:
{{
  "author": "Support Agent",
  "message": "<friendly message asking for more details>"
}}

Use this endpoint to send the response:
POST /api/tickets/{ticket_id}/respond
"""
    )


async def determine_status(ticket_id):
    await run_agent(
        ticket_id,
        "Checking ticket status",
        f"""
Update the ticket's ({ticket_id})  status.

POST /api/tickets/{ticket_id}/status

Rules:
- If priority is "High" AND the issue appears urgent → set status to "escalated".
- Otherwise → do nothing.

JSON body for status updates:
{{
  "status": "<new_status>"
}}

Valid statuses:
- open
- in progress
- closed
- escalated
"""
    )


async def determine_escalation(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if escalation is required",
        f"""
Retrieve ticket ({ticket_id})  details:

GET /api/tickets/{ticket_id}

If status == "escalated":
  → POST an automatic escalation response.

JSON body:
{{
  "author": "Support Agent",
  "message": "<context-aware escalation message>"
}}

POST to:
POST /api/tickets/{ticket_id}/respond
"""
    )


async def auto_respond(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if auto-response is needed",
        f"""
Retrieve all ({ticket_id}) responses:

GET /api/tickets/{ticket_id}/responses

If the most recent response was written by "Support Agent", do nothing.

Otherwise, POST a placeholder auto-response:

JSON body:
{{
  "author": "Support Agent",
  "message": "<friendly placeholder>"
}}

POST to:
POST /api/tickets/{ticket_id}/respond
"""
    )



# ------------------ WebSocket Listener ------------------
WS_URL = "ws://localhost:3000/ws"

async def listen_for_ticket_updates():
    print("\n------------------- Starting WebSocket Connection -------------------")
    async with websockets.connect(WS_URL) as websocket:
        print("WebSocket connection established.")
        try:
            while True:
                message = await websocket.recv()
                yield json.loads(message)
        except websockets.ConnectionClosed:
            print("WebSocket connection closed.")
        except Exception as e:
            print(f"WebSocket error: {e}")

# ------------------ Main Listener Loop ------------------
async def main_listener():
    async for message in listen_for_ticket_updates():
        ticket_id = message.get('ticketId')
        update_type = message.get('updateType')
        print(f"Processing ticket {ticket_id}, update type: {update_type}")

        if update_type == 'created':
            # Step through ticket workflow sequentially
            await determine_category(ticket_id)
            await determine_priority(ticket_id)
            await determine_response(ticket_id)
            await determine_status(ticket_id)
            await determine_escalation(ticket_id)

            # Clear context for next ticket
            clear_ticket_context(ticket_id)

        elif update_type == 'response':
            await auto_respond(ticket_id)
            clear_ticket_context(ticket_id)

# ------------------ Run in Jupyter ------------------
# Use `await` directly in a cell instead of asyncio.run
await main_listener()


------------------- Loaded Key -------------------

------------------- Starting WebSocket Connection -------------------
WebSocket connection established.
Processing ticket cdb9c751-1660-4a89-b725-e39900434113, update type: created
Categorizing ticket for ticket: cdb9c751-1660-4a89-b725-e39900434113


> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to update the category of the ticket with ID cdb9c751-1660-4a89-b725-e39900434113. The valid categories are Mechanical, Quality, Maintenance, Technical, and Awaiting Details. If there is insufficient information, I should choose "Awaiting Details". I need to use the POST method to update the category.
Observation: 1. Evaluate whether the user query can be solved by the API:

   Yes, the user query can be solved by the API. The user wants to update the category of a ticket, and the API provides an endpoint to update ticket details, including the category.

2. Generate a plan of API calls:

   1. Use the POST /a

## Basic RAG System
The follow cell sets up a basic Retrieval-Augmented Generation (RAG) retriever for the support information. This allows the agent to access relevant support documents when answering user queries, enhancing its ability to provide accurate and helpful responses. It does this by:

1. Loading the support documents from a specified directory.
2. Creating a vector store to index the documents.
3. Demonstrating how to use the retriever to get relevant information based on a user query.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

# Initialize the embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Load documents from a directory
loader = DirectoryLoader("./support-info")
docs = loader.load()

# Initialize the vector store and add documents to it
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(docs)

# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Use the retriever to find the most relevant documents for a given query
query = "Machine won't start."
relevant_docs = retriever.get_relevant_documents(query)

# Print the retrieved documents
for doc in relevant_docs:
    print(doc.page_content)